## STAC OWSLib Interoperability Demo

Goal: Show that open source OGC clients can be used to perform meaningful queries on GeoAnalytics STAC API

### About OWSLib

>OWSLib is a Python package for client programming with Open Geospatial Consortium (OGC) web service (hence OWS) interface standards, and their related content models. 

https://geopython.github.io/OWSLib

In [1]:
import os

import pandas as pd
from owslib.ogcapi.features import Features

In [2]:
STAC_URL = 'https://stac.geoanalytics.ca'
AUTH_TOKEN = os.environ['AUTH_TOKEN']

In [3]:
stac = Features(STAC_URL, headers={'cookie': f'_oauth2_proxy={AUTH_TOKEN}'})

In [4]:
def format_time_query(start, end):
    if (start is None) or (start == ''):
        start = '..'
    if (end is None) or (end == ''):
        end = '..'
    return f'{start}/{end}'

In [5]:
def format_property_query(key, val, op='='):
    return f'{key}{op}{val}'

In [6]:
def summarize_response(res):
    if (not res['type']):
        print(f'unexpected result {res}')
    features = res['features']
    count = len(features)
    rows = [extract_row(x) for x in features]
    df = pd.DataFrame(rows)
    first = df['datetime'].min()
    last = df['datetime'].max()
    print(f'Query returned {count} points from {first} to {last}')
    return df

def extract_row(row):
    return {
        'id': row['id'],
        'datetime': row['properties'] and row['properties']['datetime'],
        'day_night_flag': row['properties'] and row['properties']['modis:day_night_flag'],
        'link': row['assets']['data']['href']
    }

In [7]:
# define query variables
evi2_collection = 'modis.mod09gq.veg.evi2'
## wider bbox
#bbox_input = [-70.0,45.0,-60.0,55.0]
## narrower bbox
bbox_input = [-65.0,47.0,-63.0,50.0]


In [8]:
# query by time, and bbox
time_start = '2015-01-01' 
time_end = ''
bbox = [str(x) for x in bbox_input]
datetime = format_time_query(time_start, time_end)


res = stac.collection_items(evi2_collection, bbox=bbox, datetime=datetime, limit=5000)
df = summarize_response(res)
df.head()

Query returned 3712 points from 2015-01-01T13:55:00Z to 2020-03-08T00:00:00Z


,id,datetime,day_night_flag,link
0,MOD09GQ.A2015001.h13v04.006.2015295022015_vegevi2,2015-01-01T13:55:00Z,BOTH,https://gp.geoanalytics.ca?object=modis-terra/...
1,MOD09GQ.A2015001.h13v03.006.2015295023525_vegevi2,2015-01-01T15:30:00Z,BOTH,https://gp.geoanalytics.ca?object=modis-terra/...
2,MOD09GQ.A2015002.h13v03.006.2015295022424_vegevi2,2015-01-02T14:35:00Z,BOTH,https://gp.geoanalytics.ca?object=modis-terra/...
3,MOD09GQ.A2015002.h13v04.006.2015295020040_vegevi2,2015-01-02T14:40:00Z,DAY,https://gp.geoanalytics.ca?object=modis-terra/...
4,MOD09GQ.A2015003.h13v04.006.2015295034315_vegevi2,2015-01-03T13:45:00Z,DAY,https://gp.geoanalytics.ca?object=modis-terra/...


In [9]:
# ex. change datetime range
time_start = '2019-01-01' 
time_end = '2019-06-01'
bbox = [str(x) for x in bbox_input]
datetime = format_time_query(time_start, time_end)


res = stac.collection_items(evi2_collection, bbox=bbox, datetime=datetime, limit=5000)
df = summarize_response(res)
df.head()

Query returned 304 points from 2019-01-01T14:15:00Z to 2019-06-01T14:20:00Z


,id,datetime,day_night_flag,link
0,MOD09GQ.A2019001.h13v03.006.2019003024904_vegevi2,2019-01-01T14:15:00Z,BOTH,https://gp.geoanalytics.ca?object=modis-terra/...
1,MOD09GQ.A2019001.h13v04.006.2019003024931_vegevi2,2019-01-01T14:15:00Z,DAY,https://gp.geoanalytics.ca?object=modis-terra/...
2,MOD09GQ.A2019002.h13v03.006.2019004031930_vegevi2,2019-01-02T14:55:00Z,BOTH,https://gp.geoanalytics.ca?object=modis-terra/...
3,MOD09GQ.A2019002.h13v04.006.2019004032044_vegevi2,2019-01-02T14:55:00Z,DAY,https://gp.geoanalytics.ca?object=modis-terra/...
4,MOD09GQ.A2019003.h13v04.006.2019005030033_vegevi2,2019-01-03T14:00:00Z,DAY,https://gp.geoanalytics.ca?object=modis-terra/...


In [10]:
## define a property filter
#  ex. filter by day-night flag
prop = format_property_query('modis:day_night_flag', 'DAY')

In [11]:
# query by time, bbox, and day-night flag

res = stac.collection_items(evi2_collection, bbox=bbox, datetime=datetime, query=prop, limit=5000)
df = summarize_response(res)
df.head()

Query returned 249 points from 2019-01-01T14:15:00Z to 2019-06-01T14:20:00Z


,id,datetime,day_night_flag,link
0,MOD09GQ.A2019001.h13v04.006.2019003024931_vegevi2,2019-01-01T14:15:00Z,DAY,https://gp.geoanalytics.ca?object=modis-terra/...
1,MOD09GQ.A2019002.h13v04.006.2019004032044_vegevi2,2019-01-02T14:55:00Z,DAY,https://gp.geoanalytics.ca?object=modis-terra/...
2,MOD09GQ.A2019003.h13v04.006.2019005030033_vegevi2,2019-01-03T14:00:00Z,DAY,https://gp.geoanalytics.ca?object=modis-terra/...
3,MOD09GQ.A2019006.h13v04.006.2019008025419_vegevi2,2019-01-06T14:30:00Z,DAY,https://gp.geoanalytics.ca?object=modis-terra/...
4,MOD09GQ.A2019007.h13v04.006.2019009033553_vegevi2,2019-01-07T13:40:00Z,DAY,https://gp.geoanalytics.ca?object=modis-terra/...
